In [79]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv("airports.csv")

In [81]:
df= df[["name","latitude_deg","longitude_deg","iso_region","municipality","iata_code","score"]]

In [82]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

### CITY Data to merge with airport

In [83]:
df_c = pd.read_csv("city.csv")
df_city = df_c[["name_of_city","state_code"]]
df_city.index+= 1
df_city["city_id"]=df_city.index

/mnt/A02ECA772ECA464E/workspace/Python/jypterbook/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## Mergeing two dataset airport and city

In [84]:
df["merge"] = df["municipality"].str.upper().str.strip()
df_city["merge"] = df_city["name_of_city"].str.upper().str.strip()

/mnt/A02ECA772ECA464E/workspace/Python/jypterbook/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [85]:
df_merge = pd.merge(df, df_city, on='merge')

In [86]:
df_merge.drop_duplicates(subset='iata_code',keep='last',inplace=True)

In [87]:
df_merge.drop(['merge','municipality'], axis=1, inplace=True)
df_merge.rename(columns={'name':'airport_name','latitude_deg':'latitude','longitude_deg':'longitude','state_code':'state_id'},inplace=True)
df_merge = df_merge[['airport_name','city_id','iata_code','iso_region','latitude','longitude','score','state_id']]
df_merge.index+= 1

In [77]:

def frame_to_sql(source):
    f = open('airport.sql', 'w')
    f.write('INSERT INTO airport ('+ 'airport_id,' + str(', '.join(source.columns)) + ') VALUES ')
    f.write('\n')
    count=1
    for index, row in source.iterrows(): 
        lst = np.insert(row.values, 0, index, axis=0)
        f.write( str(tuple(lst)))
        f.write(';') if count==len(source) else f.write(',\n')
        count=count+1
        
    f.close()

In [78]:
frame_to_sql(df_merge)